## 1. Setting Up the Environment
Setting environment, including installing necessary libraries and importing dataset

In [3]:
# Installing Libraries
# !pip install obspy seisbench

In [1]:
# Importing Libraries
import os
import timeit
import random
import logging
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import h5py
import obspy
from obspy import Trace, UTCDateTime, read

import torch
import torch.nn as nn
import seisbench.models as sbm

import keras

In [2]:
# Setting Dataset Paths: Metadata
csv_path = 'dataset\\stead\\metadata.csv'
csv_path = 'D:\\Gempa 3\\dataset\\cache_seisbench\\datasets\\stead\\metadata.csv'

# Read metadata, fill noise data p-arrival value with 0
df = pd.read_csv(csv_path)
df['p_arrival_sample'] = df['p_arrival_sample'].fillna(0)
# Selection for minimum p-arrival in 11s data point position
df = df[df['p_arrival_sample']>=1100]
df

C:\Users\pc baru\AppData\Local\Temp\ipykernel_5328\2254099479.py:6: DtypeWarning: Columns (7,11,13,14,24,25,26,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,p_travel_sec,...,source_magnitude_author,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,trace_name
236301,ZQ,AAM1,HH,-26.4270,27.4230,1169.9,1111.7,autopicker,0.85,4.180000,...,NaN,NaN,0.1061,11.77,271.0522,[31.70000076 30.39999962 27. ],[[2571.]],2006-12-21 09:41:06.263000,earthquake_local,AAM1.ZQ_20061221094105_EV
236314,ZQ,AAM1,HH,-26.4270,27.4230,1169.9,1129.0,autopicker,0.78,4.770000,...,NaN,NaN,0.1281,14.22,75.5899,[15.10000038 18.39999962 13.60000038],[[3054.]],2007-01-02 01:33:18.480000,earthquake_local,AAM1.ZQ_20070102013317_EV
236327,ZQ,AAM1,HH,-26.4270,27.4230,1169.9,1111.6,autopicker,0.73,2.810000,...,NaN,NaN,0.0314,3.49,350.1123,[23.39999962 20.39999962 18.29999924],[[2667.]],2007-01-06 15:56:44.694000,earthquake_local,AAM1.ZQ_20070106155643_EV
236338,ZQ,AAM1,HH,-26.4270,27.4230,1169.9,1130.4,autopicker,0.88,3.150000,...,NaN,NaN,0.0403,4.47,69.6944,[38.90000153 39.59999847 37.09999847],[[2628.]],2007-01-08 17:33:43.346000,earthquake_local,AAM1.ZQ_20070108173342_EV
236359,ZQ,AAM1,HH,-26.4270,27.4230,1169.9,1121.9,autopicker,0.91,2.670000,...,NaN,NaN,0.0152,1.69,10.2091,[35. 34.90000153 32.29999924],[[2503.]],2007-01-13 05:43:07.951000,earthquake_local,AAM1.ZQ_20070113054306_EV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242725,AF,WDLM,HH,-26.4240,27.4220,1680.0,1124.8,autopicker,0.89,6.760000,...,NaN,NaN,0.2347,26.05,103.5824,[31.10000038 32.59999847 30.79999924],[[2041.]],2015-08-25 23:35:31.312000,earthquake_local,WDLM.AF_20150825233530_EV
1247067,IC,WMQ,HH,43.8138,87.7049,850.0,2176.0,autopicker,0.90,17.629999,...,NaN,NaN,0.2540,28.20,276.1596,[35.09999847 35.20000076 30.39999962],[[3199.]],2004-11-18 10:05:29.690000,earthquake_local,WMQ.IC_20041118100528_EV
1247068,IC,WMQ,HH,43.8138,87.7049,850.0,1485.7,autopicker,0.70,11.800000,...,NaN,NaN,0.3100,34.41,346.3289,[29.79999924 28.29999924 23.89999962],[[3199.]],2005-01-08 22:54:13.943000,earthquake_local,WMQ.IC_20050108225412_EV
1247071,IC,WMQ,HH,43.8138,87.7049,850.0,1952.8,autopicker,0.64,13.760000,...,NaN,NaN,0.1686,18.71,357.3201,[42.40000153 44.40000153 35.59999847],[[3199.]],2005-02-13 20:09:51.432000,earthquake_local,WMQ.IC_20050213200950_EV


In [3]:
# Setting Dataset Paths: Waveform
wave_path = lambda x: f'dataset\\stead\\waveform\\chunk{x}.hdf5'
wave_path = lambda x: f'C:\\Users\\pc baru\\Downloads\\chunk1\\chunk{x}.hdf5'

counting = 0
trace_names = []
chuck_name = []

# Read Waveform in 6 chunk
for i in tqdm(range(6)):
    with h5py.File(wave_path(i+1), "r") as f:
        counting += len(f['data'].keys())
        trace_names += list(f['data'].keys())
        chuck_name += [i+1] * len(f['data'].keys())
        
print("Total data:", counting)

chunkdict = {trc:chu for trc, chu in zip(trace_names, chuck_name)}
df['chunk'] = df['trace_name'].apply(lambda x: chunkdict[x])

# Just resample 200000 for each chunk
chunk_traces = pd.concat([df[df.chunk==i].iloc[:200000] for i in range(1,7)])[['chunk','trace_name','p_arrival_sample']].groupby('chunk').agg(list)
chunk_traces['length'] = chunk_traces['trace_name'].apply(lambda x: len(x))
chunk_traces

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:10<00:00,  1.67s/it]


Total data: 1265657


,trace_name,p_arrival_sample,length
chunk,,,
2,"[AAM1.ZQ_20061221094105_EV, AAM1.ZQ_2007010201...","[1111.7, 1129.0, 1111.6, 1130.4, 1121.9, 1141....",1288
3,"[B087.PB_20140713214715_EV, B093.PB_2011032601...","[1174.0, 1110.0, 2180.0, 1507.0, 1461.0, 1393....",68
4,"[CRAG.AT_20100619221659_EV, CRAG.AT_2010082816...","[1184.075, 1415.7, 1407.1, 1166.5, 1345.46, 18...",33
5,"[LLLB.CN_20000911050730_EV, MAKZ.IU_2002112812...","[1364.1, 1488.0, 1443.9, 1689.2, 1122.8, 1252....",56
6,"[PASC.CI_20120112090940_EV, PHRA.TM_2010101004...","[1103.0, 1373.1, 1151.0, 1183.0, 1430.16, 1590...",74


## 2. Data Preprocessing

In [4]:
# Interpolation function
def cstream(dt0, sr1, sr2, data):
    # Set the start time, end time, and sampling rate
    start_time = UTCDateTime.now() - dt0  # Replace with your desired start time
    end_time = UTCDateTime.now()  # Calculate the end time based on the data length and sampling rate
    sampling_rate = sr1
    st = obspy.Stream()
    
    # Create an ObsPy trace
    trace0 = obspy.Trace(data=data[:,0], header={
        'starttime': start_time,
        'endtime': end_time,
        'sampling_rate': sampling_rate
    })
    trace1 = obspy.Trace(data=data[:,1], header={
        'starttime': start_time,
        'endtime': end_time,
        'sampling_rate': sampling_rate
    })
    trace2 = obspy.Trace(data=data[:,2], header={
        'starttime': start_time,
        'endtime': end_time,
        'sampling_rate': sampling_rate
    })
    st.append(trace0)
    st.append(trace1)
    st.append(trace2)
    trace = st.interpolate(sampling_rate=sr2, method="linear")
    return np.moveaxis(np.array([
        trace[-3].data[:],
        trace[-2].data[:],
        trace[-1].data[:],
        ]), -2, -1)

In [5]:
def prep(series, mode):
    if mode=="z_norm":
        mean = np.mean(series)
        std = np.std(series)
        return (series - mean) / std
    elif mode=="min_max":
        min_val=0
        max_val=1
        series_min = np.min(series)
        series_max = np.max(series)
        return (series - series_min) * (max_val - min_val) / (series_max - series_min) + min_val
    elif mode=="mean_norm":
        mean = series.mean(0)
        return series - mean
    elif mode=="max_abs":
        max_absolute = np.max(np.abs(series))
        return series / max_absolute
    elif mode=="power_transform":
        sign = np.sign(series)
        return sign * np.abs(series) ** 0.5
    elif mode=="scaled_norm": 
        return series/1000
    elif mode=="proposed_norm": 
        return (series - series[0])/1000
    return series   

## 3. Testing PhaseNet

In [8]:
# Testing PhaseNet
model = sbm.PhaseNet.from_pretrained("original")
model = model.to('cuda')

foldername = "save_model/stead"

ARRAY = [] 
TRACE = []
sr = 100
window = 10
chunkdict = {trc:chu for trc, chu in zip(trace_names, chuck_name)}
df['chunk'] = df['trace_name'].apply(lambda x: chunkdict[x])
chunk_traces = df[['chunk','trace_name','p_arrival_sample']].groupby('chunk').agg(list)
chunk_traces['length'] = chunk_traces['trace_name'].apply(lambda x: len(x))

def prep_phase(x):
    mean = np.mean(x)
    std = np.std(x)
    return ((x) - mean) / std

def get_predict_phase(f,x,i, norm=1000):
    name = x.trace_name[i]
    data = np.array(f['data'][x.trace_name[i]][:])
    datas = [prep_phase(data[ij : ij+int(sr*window)]) for ij in range(0,6000,int(sr*window))]
    return name, datas

for chu in chunk_traces.index:
    with h5py.File(wave_path(chu), "r") as f:
        ARRAY = []
        PREDS = []
        for i in tqdm(range(chunk_traces.loc[chu].length)):
            name, data = get_predict_phase(f, chunk_traces.loc[chu], i)
            TRACE.append(name)
            ARRAY += data
        arrr = np.array(ARRAY)
        for i in tqdm(range(0,len(arrr), 1000)):
            preds = model(torch.tensor(np.moveaxis(arrr[i:i+1000], -1, -2)).to('cuda').float()).cpu().detach().numpy()
            PREDS += preds.tolist()
            
        PREDS = np.array(PREDS)
        np.save(f'{foldername}/{window}s/cont_pred_stead_phasenet_{chu}_.npy', PREDS)
            

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.57it/s]


## 4. Testing GPD

In [16]:
import torch
import torch.nn as nn
import seisbench.models as sbm

# Modeling function
def get_model_gpd(pretrained, data_point, modify=False):
    if pretrained:
        model = sbm.GPD.from_pretrained("original")
    else:
        model = sbm.GPD()

    if data_point == 80:
        model.fc1 = torch.nn.Linear(1280, 200)
    elif data_point == 400:
        model.fc1 = torch.nn.Linear(6400, 200)

    if modify:
        model.fc3 = torch.nn.Linear(200, 2)
    return model

foldername = "save_model/stead"

ARRAY = [] 
TRACE = []
sr = 20
window = 4 
chunkdict = {trc:chu for trc, chu in zip(trace_names, chuck_name)}
df['chunk'] = df['trace_name'].apply(lambda x: chunkdict[x])
chunk_traces = df[['chunk','trace_name','p_arrival_sample']].groupby('chunk').agg(list)
chunk_traces['length'] = chunk_traces['trace_name'].apply(lambda x: len(x))

model = get_model_gpd(False, int(window*sr))
# Load saved model
model.load_state_dict(torch.load(f'save_model/stead/4s/gpd_4s_20hz.pth'))
model.to('cuda')

def get_predict(f,x,i, norm=1000):
    name = x.trace_name[i]
    data = np.array(f['data'][x.trace_name[i]])[:]
    if sr!=100:
        data = cstream(60, 100, sr, data)[:-1]
    data = prep(data, 'proposed_norm')
    datas = [prep(data[ij:ij+int(window*sr)], 'proposed_norm') for ij in range(0,1200-int(window*sr))]
    return name, datas

for chu in chunk_traces.index:
    with h5py.File(wave_path(chu), "r") as f:
        PREDS = []
        for i in tqdm(range(chunk_traces.loc[chu].length)):
            name, data = get_predict(f, chunk_traces.loc[chu], i)
            TRACE.append(name)
            arrr = np.array(data)    
            preds = model(torch.tensor(np.moveaxis(arrr, -1, -2)).to('cuda').float()).cpu().detach().numpy()
            PREDS += preds.tolist()
            
        PREDS = np.array(PREDS)
        np.save(f'{foldername}/{window}s/cont_pred_stead_gpd-20hz_{chu}_.npy', PREDS)


100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:01<00:00, 49.66it/s]


## 5. Testing CNNSET

In [15]:
# Load pretrained keras model
model = keras.models.load_model(f'save_model/stead/8s/04convset_8s_20hz.h5', compile=False)
model.compile(optimizer='adam', loss='mae')

foldername = "save_model/stead"

ARRAY = [] 
TRACE = []
sr = 20
window = 8 
chunkdict = {trc:chu for trc, chu in zip(trace_names, chuck_name)}
df['chunk'] = df['trace_name'].apply(lambda x: chunkdict[x])
chunk_traces = df[['chunk','trace_name','p_arrival_sample']].groupby('chunk').agg(list)
chunk_traces['length'] = chunk_traces['trace_name'].apply(lambda x: len(x))

def get_predict(f,x,i, norm=1000):
    name = x.trace_name[i]
    data = np.array(f['data'][x.trace_name[i]])[:]
    if sr!=100:
        data = cstream(60, 100, sr, data)[:-1]
    data = prep(data, 'proposed_norm')
    datas = [prep(data[ij:ij+int(window*sr)], 'proposed_norm') for ij in range(0,1200-int(window*sr))]
    return name, datas

for chu in chunk_traces.index:
    with h5py.File(wave_path(chu), "r") as f:
        PREDS = []
        ARRAY = []
        for i in tqdm(range(chunk_traces.loc[chu].length)):
            name, data = get_predict(f, chunk_traces.loc[chu], i)
            TRACE.append(name)
            arrr = np.array(data) 
            preds = model.predict(arrr)
            PREDS += preds.tolist()
            
        PREDS = np.array(PREDS)
        np.save(f'{foldername}/{window}s/cont_pred_stead_cnnset_{chu}_.npy', PREDS)

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:19<00:00,  3.78it/s]
